### this compares two signals using a perm test and plots their wavelets

In [ ]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/")  # need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it


import mne.time_frequency
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
from ieeg.calc.fast import mean_diff
from ieeg.calc.stats import time_perm_cluster
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
from ieeg.viz.ensemble import chan_grid
from ieeg.viz.parula import parula_map

import numpy as np
# from utils import calculate_RTs, get_good_data
from src.analysis.spec.wavelet_functions import (
    load_wavelets,
    make_and_get_sig_wavelet_differences,
    load_and_get_sig_wavelet_differences,
    load_and_get_sig_wavelet_ratio_differences,
    load_multitaper,
    make_and_get_sig_multitaper_differences,
    load_and_get_sig_multitaper_differences,
    plot_mask_pages
)

import matplotlib.pyplot as plt 

from typing import List, Tuple, Optional


['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin', 'C:/Users/j

In [4]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data('GlobalLocal', root=LAB_root)

# subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110']
subjects = ['D0057']

# this is a toggle for which version to run - the one that makes the wavelets in this notebook directly, or the one that loads them
make_wavelets=False

save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', 'figs')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

maybe consider using this big ass dict from roi_analysis.ipynb to map conditions onto events

In [3]:
# # # example of how to use this with multiple conditions, even matching any value in a list. Although I only ever have two conditions of a type so not super necessary.
# # # make sure to use the correct column names and values that match with what combinedData uses.

# # congruency
# stimulus_congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25.0", "Stimulus/c75.0"],
#         "congruency": "c"
#     },
#     "Stimulus_i": {
#         "BIDS_events": ["Stimulus/i25.0", "Stimulus/i75.0"],
#         "congruency": "i"
#     }
# }

# # switch type
# stimulus_switch_type_conditions = {
#     "Stimulus_r": {
#         "BIDS_events": ["Stimulus/r25.0", "Stimulus/r75.0"],
#         "switchType": "r"
#     },
#     "Stimulus_s": {        
#         "BIDS_events": ["Stimulus/s25.0", "Stimulus/s75.0"],
#         "switchType": "s"
#     }
# }

# # #  ir vs is
# # output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# # output_names_conditions = {
# #     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
# #         "congruency": "i",
# #         "switchType": "r"
# #     },
# #     "Stimulus_is_fixationCrossBase_1sec_mirror": {
# #         "congruency": "i",
# #         "switchType": "s"
# #     }
# # }

# # #  cr vs cs
# # output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]
# # output_names_conditions = {
# #     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
# #         "congruency": "c",
# #         "switchType": "r"
# #     },
# #     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
# #         "congruency": "c",
# #         "switchType": "s"
# #     }
# # }

# # #  is vs cs
# # output_names = ["Stimulus_cs_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# # output_names_conditions = {
# #     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
# #         "congruency": "c",
# #         "switchType": "s"
# #     },
# #     "Stimulus_is_fixationCrossBase_1sec_mirror": {
# #         "congruency": "i",
# #         "switchType": "s"
# #     }
# # }

# # #  ir vs cr
# # output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_ir_fixationCrossBase_1sec_mirror"]
# # output_names_conditions = {
# #     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
# #         "congruency": "c",
# #         "switchType": "r"
# #     },
# #     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
# #         "congruency": "i",
# #         "switchType": "r"
# #     }
# # }

# # # # main effect interaction effects (run this with the anova code. Ugh make everything more modular later.)

# stimulus_main_effect_conditions = {
#     "Stimulus_ir": {
#         "BIDS_events": ["Stimulus/i25.0/r25.0", "Stimulus/i25.0/r75.0", "Stimulus/i75.0/r25.0", "Stimulus/i75.0/r75.0"],
#         "congruency": "i",
#         "switchType": "r"
#     },
#     "Stimulus_is": {
#         "BIDS_events": ["Stimulus/i25.0/s25.0", "Stimulus/i25.0/s75.0", "Stimulus/i75.0/s25.0", "Stimulus/i75.0/s75.0"],
#         "congruency": "i",
#         "switchType": "s"
#     },
#     "Stimulus_cr": {
#         "BIDS_events": ["Stimulus/c25.0/r25.0", "Stimulus/c25.0/r75.0", "Stimulus/c75.0/r25.0", "Stimulus/c75.0/r75.0"],
#         "congruency": "c",
#         "switchType": "r"
#     },
#     "Stimulus_cs": {
#         "BIDS_events": ["Stimulus/c25.0/s25.0", "Stimulus/c25.0/s75.0", "Stimulus/c75.0/s25.0", "Stimulus/c75.0/s75.0"],
#         "congruency": "c",
#         "switchType": "s"
#     }
# }

# # # block interaction contrasts for lwpc

# stimulus_lwpc_conditions = {
#     "Stimulus_c25": {
#         "BIDS_events": "Stimulus/c25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     },
#     "Stimulus_c75": {
#         "BIDS_events": "Stimulus/c75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     },
#     "Stimulus_i25": {
#         "BIDS_events": "Stimulus/i25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     },
#     "Stimulus_i75": {
#         "BIDS_events": "Stimulus/i75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     }
# }

# # # block interaction contrasts for lwps

# stimulus_lwps_conditions = {
#     "Stimulus_s25": {
#         "BIDS_events": "Stimulus/s25.0",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_s75": {
#         "BIDS_events": "Stimulus/s75.0",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_r25": {
#         "BIDS_events": "Stimulus/r25.0",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_r75": {
#         "BIDS_events": "Stimulus/r75.0",
#         "switchType": "r",
#         "switchProportion": "75%"
#     }
# }

# # all 16 trial types
# stimulus_experiment_conditions = {
#     "Stimulus_i25s25": {
#         "BIDS_events": "Stimulus/i25.0/s25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_i25s75": {
#         "BIDS_events": "Stimulus/i25.0/s75.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_i75s25": {
#         "BIDS_events": "Stimulus/i75.0/s25.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_i75s75": {
#         "BIDS_events": "Stimulus/i75.0/s75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_i25r25": {
#         "BIDS_events": "Stimulus/i25.0/r25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_i25r75": {
#         "BIDS_events": "Stimulus/i25.0/r75.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Stimulus_i75r25": {
#         "BIDS_events": "Stimulus/i75.0/r25.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_i75r75": {
#         "BIDS_events": "Stimulus/i75.0/r75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Stimulus_c25s25": {
#         "BIDS_events": "Stimulus/c25.0/s25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_c25s75": {
#         "BIDS_events": "Stimulus/c25.0/s75.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_c75s25": {
#         "BIDS_events": "Stimulus/c75.0/s25.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_c75s75": {
#         "BIDS_events": "Stimulus/c75.0/s75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_c25r25": {
#         "BIDS_events": "Stimulus/c25.0/r25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_c25r75": {
#         "BIDS_events": "Stimulus/c25.0/r75.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Stimulus_c75r25": {
#         "BIDS_events": "Stimulus/c75.0/r25.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_c75r75": {
#         "BIDS_events": "Stimulus/c75.0/r75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     }
# }

# # stimulus details
# stimulus_conditions = {
#     "Stimulus_bigSsmallHtaskG": {
#         "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
#         "bigLetter": "s",
#         "smallLetter": "h",
#         "task": "g"
#     },
#     "Stimulus_bigSsmallHtaskL": {
#         "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
#         "bigLetter": "s",
#         "smallLetter": "h",
#         "task": "l"
#     },
#     "Stimulus_bigSsmallStaskG": {
#         "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
#         "bigLetter": "s",
#         "smallLetter": "s",
#         "task": "g"
#     },
#     "Stimulus_bigSsmallStaskL": {
#         "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
#         "bigLetter": "s",
#         "smallLetter": "s",
#         "task": "l"
#     },
#     "Stimulus_bigHsmallHtaskG": {
#         "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
#         "bigLetter": "h",
#         "smallLetter": "h",
#         "task": "g"
#     },
#     "Stimulus_bigHsmallHtaskL": {
#         "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
#         "bigLetter": "h",
#         "smallLetter": "h",
#         "task": "l"
#     },
#     "Stimulus_bigHsmallStaskG": {
#         "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
#         "bigLetter": "h",
#         "smallLetter": "s",
#         "task": "g"
#     },
#     "Stimulus_bigHsmallStaskL": {
#         "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
#         "bigLetter": "h",
#         "smallLetter": "s",
#         "task": "l"
#     }
# }


# # big letter details
# stimulus_big_letter_conditions = {
#     "Stimulus_bigS": {
#         "BIDS_events": "Stimulus/BigLetters",
#         "bigLetter": "s",
#     },
#     "Stimulus_bigH": {
#         "BIDS_events": "Stimulus/BigLetterh",
#         "bigLetter": "h",
#     }
# }

# # small letter details
# stimulus_small_letter_conditions = {
#     "Stimulus_smallS": {
#         "BIDS_events": "Stimulus/SmallLetters",
#         "smallLetter": "s",
#     },
#     "Stimulus_smallH": {
#         "BIDS_events": "Stimulus/SmallLetterh",
#         "smallLetter": "h",
#     }
# }

# # task details
# stimulus_task_conditions = {
#     "Stimulus_taskG": {
#         "BIDS_events": "Stimulus/Taskg",
#         "task": "g",
#     },
#     "Stimulus_taskL": {
#         "BIDS_events": "Stimulus/Taskl",
#         "task": "l",
#     }
# }

# # congruency
# response_congruency_conditions = {
#     "Response_c": {
#         "BIDS_events": ["Response/c25.0", "Response/c75.0"],
#         "congruency": "c"
#     },
#     "Response_i": {
#         "BIDS_events": ["Response/i25.0", "Response/i75.0"],
#         "congruency": "i"
#     }
# }

# # switch type
# response_switch_type_conditions = {
#     "Response_r": {
#         "BIDS_events": ["Response/r25.0", "Response/r75.0"],
#         "switchType": "r"
#     },
#     "Response_s": {        
#         "BIDS_events": ["Response/s25.0", "Response/s75.0"],
#         "switchType": "s"
#     }
# }

# response_experiment_conditions = {
#     "Response_i25s25": {
#         "BIDS_events": "Response/i25.0/s25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Response_i25s75": {
#         "BIDS_events": "Response/i25.0/s75.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Response_i75s25": {
#         "BIDS_events": "Response/i75.0/s25.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Response_i75s75": {
#         "BIDS_events": "Response/i75.0/s75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Response_i25r25": {
#         "BIDS_events": "Response/i25.0/r25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Response_i25r75": {
#         "BIDS_events": "Response/i25.0/r75.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Response_i75r25": {
#         "BIDS_events": "Response/i75.0/r25.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Response_i75r75": {
#         "BIDS_events": "Response/i75.0/r75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Response_c25s25": {
#         "BIDS_events": "Response/c25.0/s25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Response_c25s75": {
#         "BIDS_events": "Response/c25.0/s75.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Response_c75s25": {
#         "BIDS_events": "Response/c75.0/s25.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Response_c75s75": {
#         "BIDS_events": "Response/c75.0/s75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Response_c25r25": {
#         "BIDS_events": "Response/c25.0/r25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Response_c25r75": {
#         "BIDS_events": "Response/c25.0/r75.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Response_c75r25": {
#         "BIDS_events": "Response/c75.0/r25.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Response_c75r75": {
#         "BIDS_events": "Response/c75.0/r75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     }
# }

# response_conditions = {
#     "Response_bigSsmallHtaskG": {
#         "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
#         "bigLetter": "s",
#         "smallLetter": "h",
#         "task": "g"
#     },
#     "Response_bigSsmallHtaskL": {
#         "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
#         "bigLetter": "s",
#         "smallLetter": "h",
#         "task": "l"
#     },
#     "Response_bigSsmallStaskG": {
#         "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
#         "bigLetter": "s",
#         "smallLetter": "s",
#         "task": "g"
#     },
#     "Response_bigSsmallStaskL": {
#         "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
#         "bigLetter": "s",
#         "smallLetter": "s",
#         "task": "l"
#     },
#     "Response_bigHsmallHtaskG": {
#         "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
#         "bigLetter": "h",
#         "smallLetter": "h",
#         "task": "g"
#     },
#     "Response_bigHsmallHtaskL": {
#         "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
#         "bigLetter": "h",
#         "smallLetter": "h",
#         "task": "l"
#     },
#     "Response_bigHsmallStaskG": {
#         "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
#         "bigLetter": "h",
#         "smallLetter": "s",
#         "task": "g"
#     },
#     "Response_bigHsmallStaskL": {
#         "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
#         "bigLetter": "h",
#         "smallLetter": "s",
#         "task": "l"
#     }
# }


# # big letter details
# response_big_letter_conditions = {
#     "Response_bigS": {
#         "BIDS_events": "Response/BigLetters",
#         "bigLetter": "s",
#     },
#     "Response_bigH": {
#         "BIDS_events": "Response/BigLetterh",
#         "bigLetter": "h",
#     }
# }

# # small letter details
# response_small_letter_conditions = {
#     "Response_smallS": {
#         "BIDS_events": "Response/SmallLetters",
#         "smallLetter": "s",
#     },
#     "Response_smallH": {
#         "BIDS_events": "Response/SmallLetterh",
#         "smallLetter": "h",
#     }
# }

# # task details
# response_task_conditions = {
#     "Response_taskG": {
#         "BIDS_events": "Response/Taskg",
#         "task": "g",
#     },
#     "Response_taskL": {
#         "BIDS_events": "Response/Taskl",
#         "task": "l",
#     }
# }

actually let's just use this small, simple dict for now 2/28

In [5]:
# Mapping condition names to their corresponding output names and event lists
conditions_and_output_names_and_events = {
    'incongruent': {
        'output_name': 'Stimulus_i25and75_fixationCrossBase_0.5sec',
        'events': ['Stimulus/i75.0', 'Stimulus/i25.0']
    },
    'congruent': {
        'output_name': 'Stimulus_c25and75_fixationCrossBase_0.5sec',
        'events': ['Stimulus/c75.0', 'Stimulus/c25.0']
    },
    'switch': {
        'output_name': 'Stimulus_s25and75_fixationCrossBase_0.5sec',
        'events': ['Stimulus/i75.0/s25.0', 'Stimulus/i25.0/s75.0', 'Stimulus/c75.0/s25.0', 'Stimulus/c25.0/s75.0']
    },
    'repeat': {
        'output_name': 'Stimulus_r25and75_fixationCrossBase_0.5sec',
        'events': ['Stimulus/i75.0/r25.0', 'Stimulus/i25.0/r75.0', 'Stimulus/c75.0/r25.0', 'Stimulus/c25.0/r75.0']
    }
}


make or load wavelets, and get their significant cluster differences

In [15]:
times = [-0.5, 1.5]

if make_wavelets:
    # For making wavelets, extract events directly from the configuration dictionary.
    incongruent_events = conditions_and_output_names_and_events['incongruent']['events']
    congruent_events   = conditions_and_output_names_and_events['congruent']['events']
    switch_events      = conditions_and_output_names_and_events['switch']['events']
    repeat_events      = conditions_and_output_names_and_events['repeat']['events']

    sig_wavelet_differences_per_subject = {}

    for sub in subjects:

        sig_wavelet_differences_per_subject[sub] = {}
        # Preprocess and compute average reaction time (if desired)
        good = get_good_data(sub, layout)
        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)
        print(f"Subject {sub} average RT: {avg_RT}")

        # do inc-con, and also switch-repeat
        congruency_mask, congruency_pvals = make_and_get_sig_wavelet_differences(
            sub, layout, incongruent_events, congruent_events, times,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)
        switch_type_mask, switch_type_pvals = make_and_get_sig_wavelet_differences(
            sub, layout, switch_events, repeat_events, times,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)

        sig_wavelet_differences_per_subject[sub]['congruency'] = (congruency_mask, congruency_pvals)
        sig_wavelet_differences_per_subject[sub]['switch_type'] = (switch_type_mask, switch_type_pvals)

else:
    # For loading precomputed wavelets, use output names from the configuration.
    # Here, 'rescaled' toggles whether to load baseline-corrected (rescaled) TFRs.
    rescaled = False  # Set to True if you wish to use baseline-corrected (dB scaled) wavelets

    sig_wavelet_differences_per_subject = {}

    for sub in subjects:

        sig_wavelet_differences_per_subject[sub] = {}
        good = get_good_data(sub, layout)
        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)
        print(f"Subject {sub} average RT: {avg_RT}")

        # do inc-con, and also switch-repeat
        congruency_mask, congruency_pvals = load_and_get_sig_wavelet_differences(
            sub, layout,
            conditions_and_output_names_and_events['incongruent']['output_name'],
            conditions_and_output_names_and_events['congruent']['output_name'],
            rescaled,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)
        switch_type_mask, switch_type_pvals = load_and_get_sig_wavelet_differences(
            sub, layout,
            conditions_and_output_names_and_events['switch']['output_name'],
            conditions_and_output_names_and_events['repeat']['output_name'],
            rescaled,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)

        sig_wavelet_differences_per_subject[sub]['congruency'] = (congruency_mask, congruency_pvals)
        sig_wavelet_differences_per_subject[sub]['switch_type'] = (switch_type_mask, switch_type_pvals)

        # plot the sig cluster masks
        # Assume mask has shape (n_channels, ...) and you have channel names:
        # For example, you can get channel names from one of your TFR objects.
        # (Here we load one TFR to extract channel names.)
        incongruent_spec = load_wavelets(sub, layout, conditions_and_output_names_and_events['incongruent']['output_name'], rescaled)
        congruent_spec = load_wavelets(sub, layout, conditions_and_output_names_and_events['congruent']['output_name'], rescaled)
        switch_spec = load_wavelets(sub, layout, conditions_and_output_names_and_events['switch']['output_name'], rescaled)
        repeat_spec = load_wavelets(sub, layout, conditions_and_output_names_and_events['repeat']['output_name'], rescaled)

        ch_names = incongruent_spec.ch_names  # list of channel names

        # Now plot the mask pages:
        congruency_mask_pages = plot_mask_pages(congruency_mask,
                            incongruent_spec.ch_names,
                            times=incongruent_spec.times,
                            freqs=incongruent_spec.freqs,
                            channels_per_page=60,
                            grid_shape=(6, 10),
                            cmap=parula_map,
                            title_prefix=f"{sub} ",
                            log_freq=True,
                            show=False)
        
        switch_type_mask_pages = plot_mask_pages(switch_type_mask,
                            switch_spec.ch_names,
                            times=switch_spec.times,
                            freqs=switch_spec.freqs,
                            channels_per_page=60,
                            grid_shape=(6, 10),
                            cmap=parula_map,
                            title_prefix=f"{sub} ",
                            log_freq=True,
                            show=False)

        # Save each page as a separate figure file:
        for i, fig in enumerate(congruency_mask_pages):
            if rescaled:
                fig_name = f"{sub}_inc-con_sig_wavelet_clusters_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_inc-con_sig_wavelet_clusters_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        for i, fig in enumerate(switch_type_mask_pages):
            if rescaled:
                fig_name = f"{sub}_switch-repeat_sig_wavelet_clusters_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_switch-repeat_sig_wavelet_clusters_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        # get the mean differences themselves and plot them
        mean_diff_inc_vs_con = mean_diff(incongruent_spec._data, congruent_spec._data, axis=0)
        mean_diff_switch_vs_repeat = mean_diff(switch_spec._data, repeat_spec._data, axis=0)

        # Now, plot the mean differences directly:
        congruency_mean_diff_pages = plot_mask_pages(
            mean_diff_inc_vs_con,
            incongruent_spec.ch_names,
            times=incongruent_spec.times,
            freqs=incongruent_spec.freqs,
            grid_shape=(6, 10),
            cmap=parula_map,  # play with color maps
            title_prefix=f"{sub} Mean Inc-Con Diff: ",
            log_freq=True,
            show=False
        )

        switch_type_mean_diff_pages = plot_mask_pages(
            mean_diff_switch_vs_repeat,
            switch_spec.ch_names,
            times=switch_spec.times,
            freqs=switch_spec.freqs,
            grid_shape=(6, 10),
            cmap=parula_map,  # play with color maps
            title_prefix=f"{sub} Mean switch-repeat Diff: ",
            log_freq=True,
            show=False
        )

        # Save each page as a separate figure file:
        for i, fig in enumerate(congruency_mean_diff_pages):
            if rescaled:
                fig_name = f"{sub}_inc-con_mean_diff_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_inc-con_mean_diff_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        # Save each page as a separate figure file:
        for i, fig in enumerate(switch_type_mean_diff_pages):
            if rescaled:
                fig_name = f"{sub}_switch-repeat_mean_diff_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_switch-repeat_mean_diff_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)


Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

<RawEDF | sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf, 178 x 8243200 (4025.0 s), ~201 kB, data not loaded>
outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
outlier round 2 channels: ['RAMT8', 'RPI16', 'LAMT12']
Reading 0 ... 3433906  =      0.000 ...  1676.712 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Subject D0057 average RT: 1.065918000000238
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_i25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    2.3s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    4.2s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    6.2s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    8.2s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:   10.1s
[Parallel(n_jobs=1)]: Done   6 tasks      | elapsed:   12.1s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:   14.0s
[Parallel(n_jobs=1)]: Done   8 tasks      | elapsed:   15.9s
[Parallel(n_jobs=1)]: Done   9 tasks      | elapsed:   17.8s
[Parallel(n_jobs=1)]: Done  10 tasks      | elapsed:   19.7s
[Parallel(n_jobs=1)]: Done  11 tasks      | elapsed:   21.7s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:   23.7s
[Parallel(n_jobs=1)]: Done  13 tasks      | elapsed:   25.7s
[Parallel(n_jobs=1)]: Done  14 tasks      | elapsed:   27.8s
[Parallel(n_jobs=1)]: Done  15 tasks      | elapsed:   29.8s
[Parallel(n_jobs=1)]: Done  16 tasks      | elapsed:   31.7s
[Parallel(n_jobs=1)]: Do

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_s25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    2.0s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    3.2s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    4.5s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    5.8s
[Parallel(n_jobs=1)]: Done   6 tasks      | elapsed:    7.2s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:    8.4s
[Parallel(n_jobs=1)]: Done   8 tasks      | elapsed:    9.5s
[Parallel(n_jobs=1)]: Done   9 tasks      | elapsed:   10.7s
[Parallel(n_jobs=1)]: Done  10 tasks      | elapsed:   11.7s
[Parallel(n_jobs=1)]: Done  11 tasks      | elapsed:   12.8s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:   13.9s
[Parallel(n_jobs=1)]: Done  13 tasks      | elapsed:   14.9s
[Parallel(n_jobs=1)]: Done  14 tasks      | elapsed:   16.0s
[Parallel(n_jobs=1)]: Done  15 tasks      | elapsed:   17.0s
[Parallel(n_jobs=1)]: Done  16 tasks      | elapsed:   17.9s
[Parallel(n_jobs=1)]: Do

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_i25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_s25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Saved figure: D0057_inc-con_sig_wavelet_clusters_Stimulus_i25and75_fixationCrossBase_0.5sec-Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_page_1.jpg
Saved figure: D0057_inc-con_sig_wavelet_clusters_Stimulus_i25and75_fixationCrossBase_0.5sec-Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_page_2.jpg
Saved figure: D0057_inc-con_sig_wavelet_clusters

hmmm let's try doing a ratio instead of the mean_diff

In [16]:
times = [-0.5, 1.5]

# For loading precomputed wavelets, use output names from the configuration.
# Here, 'rescaled' toggles whether to load baseline-corrected (rescaled) TFRs.
rescaled = False  # Set to True if you wish to use baseline-corrected (dB scaled) wavelets

sig_wavelet_differences_per_subject = {}

for sub in subjects:

    sig_wavelet_differences_per_subject[sub] = {}
    good = get_good_data(sub, layout)
    RTs, skipped = calculate_RTs(good)
    avg_RT = np.median(RTs)
    print(f"Subject {sub} average RT: {avg_RT}")

    # Use the ratio function to load TFRs, compute their ratio in dB, and test significance
    congruency_ratio_mask, congruency_ratio_pvals, congruency_ratio_spec = load_and_get_sig_wavelet_ratio_differences(
        sub, layout,
        conditions_and_output_names_and_events['incongruent']['output_name'],
        conditions_and_output_names_and_events['congruent']['output_name'],
        rescaled,
        stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1
    )

    switch_type_ratio_mask, switch_type_ratio_pvals, switch_type_ratio_spec = load_and_get_sig_wavelet_ratio_differences(
        sub, layout,
        conditions_and_output_names_and_events['switch']['output_name'],
        conditions_and_output_names_and_events['repeat']['output_name'],
        rescaled,
        stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1
    )


    sig_wavelet_differences_per_subject[sub]['congruency'] = (congruency_ratio_mask, congruency_ratio_pvals)
    sig_wavelet_differences_per_subject[sub]['switch_type'] = (switch_type_ratio_mask, switch_type_ratio_pvals)

    # Plot the significant cluster mask pages (for the ratio test)
    congruency_ratio_mask_pages = plot_mask_pages(
        congruency_ratio_mask,
        congruency_ratio_spec.ch_names,
        times=congruency_ratio_spec.times,
        freqs=congruency_ratio_spec.freqs,
        channels_per_page=60,
        grid_shape=(6, 10),
        cmap=parula_map,
        title_prefix=f"{sub} ",
        log_freq=True,
        show=False
    )

    switch_type_ratio_mask_pages = plot_mask_pages(
        switch_type_ratio_mask,
        switch_type_ratio_spec.ch_names,
        times=switch_type_ratio_spec.times,
        freqs=switch_type_ratio_spec.freqs,
        channels_per_page=60,
        grid_shape=(6, 10),
        cmap=parula_map,
        title_prefix=f"{sub} ",
        log_freq=True,
        show=False
    )

    for i, fig in enumerate(congruency_ratio_mask_pages):
        if rescaled:
            fig_name = f"{sub}_inc_vs_con_sig_wavelet_clusters_ratio_{conditions_and_output_names_and_events['incongruent']['output_name']}_vs_{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
        else:
            fig_name = f"{sub}_inc_vs_con_sig_wavelet_clusters_ratio_{conditions_and_output_names_and_events['incongruent']['output_name']}_vs_{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
        fig_pathname = os.path.join(save_dir, fig_name)
        fig.savefig(fig_pathname, bbox_inches='tight')
        print("Saved figure:", fig_name)

    for i, fig in enumerate(switch_type_ratio_mask_pages):
        if rescaled:
            fig_name = f"{sub}_switch_vs_repeat_sig_wavelet_clusters_ratio_{conditions_and_output_names_and_events['switch']['output_name']}_vs_{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_page_{i+1}.jpg"
        else:
            fig_name = f"{sub}_switch_vs_repeat_sig_wavelet_clusters_ratio_{conditions_and_output_names_and_events['switch']['output_name']}_vs_{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_page_{i+1}.jpg"
        fig_pathname = os.path.join(save_dir, fig_name)
        fig.savefig(fig_pathname, bbox_inches='tight')
        print("Saved figure:", fig_name)

    # also plot the congruency and switch type ratio spectrograms themselves
    avg_congruency_ratio_spec = congruency_ratio_spec.average(lambda x: np.nanmean(x, axis=0), copy=True)

    # Use plot_mask_pages instead of chan_grid
    congruency_ratio_spec_pages = plot_mask_pages(
        avg_congruency_ratio_spec.data,
        avg_congruency_ratio_spec.ch_names,
        times=avg_congruency_ratio_spec.times,
        freqs=avg_congruency_ratio_spec.freqs,
        channels_per_page=60,
        grid_shape=(6, 10),
        cmap=parula_map,
        title_prefix=f"{sub} Ratio (inc/con): ",
        log_freq=True,
        show=False,
        colorbar_range=(-2, 2)  # Same as your vmin/vmax in chan_grid
    )

    # Save the figures
    for i, fig in enumerate(congruency_ratio_spec_pages):
        if rescaled:
            fig_name = f"{sub}_inc_vs_con_wavelet_ratio_spec_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_tall_page_{i+1}.jpg"
        else:
            fig_name = f"{sub}_inc_vs_con_wavelet_ratio_spec_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_tall_page_{i+1}.jpg"
        
        fig_pathname = os.path.join(save_dir, fig_name)
        fig.savefig(fig_pathname, bbox_inches='tight')
        print("Saved figure:", fig_name)

    # For switch type ratio
    avg_switch_type_ratio_spec = switch_type_ratio_spec.average(lambda x: np.nanmean(x, axis=0), copy=True)

    switch_type_ratio_spec_pages = plot_mask_pages(
        avg_switch_type_ratio_spec.data,
        avg_switch_type_ratio_spec.ch_names,
        times=avg_switch_type_ratio_spec.times,
        freqs=avg_switch_type_ratio_spec.freqs,
        channels_per_page=60,
        grid_shape=(6, 10),
        cmap=parula_map,
        title_prefix=f"{sub} Ratio (switch/repeat): ",
        log_freq=True,
        show=False,
        colorbar_range=(-2, 2)
    )

    for i, fig in enumerate(switch_type_ratio_spec_pages):
        if rescaled:
            fig_name = f"{sub}_switch_vs_repeat_wavelet_ratio_spec_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_tall_page_{i+1}.jpg"
        else:
            fig_name = f"{sub}_switch_vs_repeat_wavelet_ratio_spec_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_tall_page_{i+1}.jpg"
        
        fig_pathname = os.path.join(save_dir, fig_name)
        fig.savefig(fig_pathname, bbox_inches='tight')
        print("Saved figure:", fig_name)

    # using chan_grid below, maybe use this for plots instead. It's wider than the plots from plot_mask_pages. 

    # figs = chan_grid(avg_congruency_ratio_spec, size=(20, 10), vmin=-2, vmax=2, cmap=parula_map, show=False)
    # for i, fig in enumerate(figs):
    #     if rescaled:
    #         fig_name = f"{sub}_inc_vs_con_wavelet_ratio_spec_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
    #     else:
    #         fig_name = f"{sub}_inc_vs_con_wavelet_ratio_spec_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
    #         fig_pathname = os.path.join(save_dir, fig_name)
    #         fig.savefig(fig_pathname, bbox_inches='tight')
    #         print("Saved figure:", fig_name)

    # avg_switch_type_ratio_spec = switch_type_ratio_spec.average(lambda x: np.nanmean(x, axis=0), copy=True)
    # figs = chan_grid(avg_switch_type_ratio_spec, size=(20, 10), vmin=-2, vmax=2, cmap=parula_map, show=False)
    # for i, fig in enumerate(figs):
    #     if rescaled:
    #         fig_name = f"{sub}_switch_vs_repeat_wavelet_ratio_spec_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_page_{i+1}.jpg"
    #     else:
    #         fig_name = f"{sub}_switch_vs_repeat_wavelet_ratio_spec_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_page_{i+1}.jpg"
    #         fig_pathname = os.path.join(save_dir, fig_name)
    #         fig.savefig(fig_pathname, bbox_inches='tight')
    #         print("Saved figure:", fig_name)


Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected


...

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_s25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    2.0s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    3.0s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    4.1s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    5.1s
[Parallel(n_jobs=1)]: Done   6 tasks      | elapsed:    6.1s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:    7.1s
[Parallel(n_jobs=1)]: Done   8 tasks      | elapsed:    8.0s
[Parallel(n_jobs=1)]: Done   9 tasks      | elapsed:    9.1s
[Parallel(n_jobs=1)]: Done  10 tasks      | elapsed:   10.1s
[Parallel(n_jobs=1)]: Done  11 tasks      | elapsed:   11.0s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:   12.0s
[Parallel(n_jobs=1)]: Done  13 tasks      | elapsed:   13.0s
[Parallel(n_jobs=1)]: Done  14 tasks      | elapsed:   14.0s
[Parallel(n_jobs=1)]: Done  15 tasks      | elapsed:   15.1s
[Parallel(n_jobs=1)]: Done  16 tasks      | elapsed:   16.0s
[Parallel(n_jobs=1)]: Do

Saved figure: D0057_inc_vs_con_sig_wavelet_clusters_ratio_Stimulus_i25and75_fixationCrossBase_0.5sec_vs_Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_page_1.jpg
Saved figure: D0057_inc_vs_con_sig_wavelet_clusters_ratio_Stimulus_i25and75_fixationCrossBase_0.5sec_vs_Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_page_2.jpg
Saved figure: D0057_inc_vs_con_sig_wavelet_clusters_ratio_Stimulus_i25and75_fixationCrossBase_0.5sec_vs_Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_page_3.jpg
Saved figure: D0057_switch_vs_repeat_sig_wavelet_clusters_ratio_Stimulus_s25and75_fixationCrossBase_0.5sec_vs_Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected_page_1.jpg
Saved figure: D0057_switch_vs_repeat_sig_wavelet_clusters_ratio_Stimulus_s25and75_fixationCrossBase_0.5sec_vs_Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected_page_2.jpg
Saved figure: D0057_switch_vs_repeat_sig_wavelet_clusters_ratio_Stimulus_s25and75_fixationCrossBase_0.5sec_vs_Stimulus_r25and75_fixationCr

c:\Users\jz421\Desktop\GlobalLocal\wavelet_functions.py:473: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  Whether to use logarithmic scaling for the frequency axis.


Saved figure: D0057_inc_vs_con_wavelet_ratio_spec_Stimulus_i25and75_fixationCrossBase_0.5sec-Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_tall_page_1.jpg
Saved figure: D0057_inc_vs_con_wavelet_ratio_spec_Stimulus_i25and75_fixationCrossBase_0.5sec-Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_tall_page_2.jpg
Saved figure: D0057_inc_vs_con_wavelet_ratio_spec_Stimulus_i25and75_fixationCrossBase_0.5sec-Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_tall_page_3.jpg
Saved figure: D0057_switch_vs_repeat_wavelet_ratio_spec_Stimulus_s25and75_fixationCrossBase_0.5sec-Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected_tall_page_1.jpg
Saved figure: D0057_switch_vs_repeat_wavelet_ratio_spec_Stimulus_s25and75_fixationCrossBase_0.5sec-Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected_tall_page_2.jpg
Saved figure: D0057_switch_vs_repeat_wavelet_ratio_spec_Stimulus_s25and75_fixationCrossBase_0.5sec-Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected_tall_page_3.jp

I'm gonna try aaron's suggestion of ignoring adjacency for the frequency column too (the only difference between this and the above code is that I do ignore_adjacency=(1,2) instead of just 1 here). This ignores clusters across frequency.

In [4]:
times = [-0.5, 1.5]

if make_wavelets:
    # For making wavelets, extract events directly from the configuration dictionary.
    incongruent_events = conditions_and_output_names_and_events['incongruent']['events']
    congruent_events   = conditions_and_output_names_and_events['congruent']['events']
    switch_events      = conditions_and_output_names_and_events['switch']['events']
    repeat_events      = conditions_and_output_names_and_events['repeat']['events']

    sig_wavelet_differences_per_subject = {}

    for sub in subjects:

        sig_wavelet_differences_per_subject[sub] = {}
        # Preprocess and compute average reaction time (if desired)
        good = get_good_data(sub, layout)
        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)
        print(f"Subject {sub} average RT: {avg_RT}")

        # do inc-con, and also switch-repeat
        congruency_mask, congruency_pvals = make_and_get_sig_wavelet_differences(
            sub, layout, incongruent_events, congruent_events, times,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=(1,2), n_perm=100, n_jobs=1)
        switch_type_mask, switch_type_pvals = make_and_get_sig_wavelet_differences(
            sub, layout, switch_events, repeat_events, times,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=(1,2), n_perm=100, n_jobs=1)

        sig_wavelet_differences_per_subject[sub]['congruency'] = (congruency_mask, congruency_pvals)
        sig_wavelet_differences_per_subject[sub]['switch_type'] = (switch_type_mask, switch_type_pvals)

else:
    # For loading precomputed wavelets, use output names from the configuration.
    # Here, 'rescaled' toggles whether to load baseline-corrected (rescaled) TFRs.
    rescaled = False  # Set to True if you wish to use baseline-corrected (dB scaled) wavelets

    sig_wavelet_differences_per_subject = {}

    for sub in subjects:

        sig_wavelet_differences_per_subject[sub] = {}
        good = get_good_data(sub, layout)
        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)
        print(f"Subject {sub} average RT: {avg_RT}")

        # do inc-con, and also switch-repeat
        congruency_mask, congruency_pvals = load_and_get_sig_wavelet_differences(
            sub, layout,
            conditions_and_output_names_and_events['incongruent']['output_name'],
            conditions_and_output_names_and_events['congruent']['output_name'],
            rescaled,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=(1,2), n_perm=100, n_jobs=1)
        switch_type_mask, switch_type_pvals = load_and_get_sig_wavelet_differences(
            sub, layout,
            conditions_and_output_names_and_events['switch']['output_name'],
            conditions_and_output_names_and_events['repeat']['output_name'],
            rescaled,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=(1,2), n_perm=100, n_jobs=1)

        sig_wavelet_differences_per_subject[sub]['congruency'] = (congruency_mask, congruency_pvals)
        sig_wavelet_differences_per_subject[sub]['switch_type'] = (switch_type_mask, switch_type_pvals)

        # plot the sig cluster masks
        # Assume mask has shape (n_channels, ...) and you have channel names:
        # For example, you can get channel names from one of your TFR objects.
        # (Here we load one TFR to extract channel names.)
        incongruent_spec = load_wavelets(sub, layout, conditions_and_output_names_and_events['incongruent']['output_name'], rescaled)
        congruent_spec = load_wavelets(sub, layout, conditions_and_output_names_and_events['congruent']['output_name'], rescaled)
        switch_spec = load_wavelets(sub, layout, conditions_and_output_names_and_events['switch']['output_name'], rescaled)
        repeat_spec = load_wavelets(sub, layout, conditions_and_output_names_and_events['repeat']['output_name'], rescaled)

        ch_names = incongruent_spec.ch_names  # list of channel names

        # Now plot the mask pages:
        congruency_mask_pages = plot_mask_pages(congruency_mask,
                            incongruent_spec.ch_names,
                            times=incongruent_spec.times,
                            freqs=incongruent_spec.freqs,
                            channels_per_page=60,
                            grid_shape=(6, 10),
                            cmap='gray',
                            title_prefix=f"{sub} ",
                            log_freq=True,
                            show=False)
        
        switch_type_mask_pages = plot_mask_pages(switch_type_mask,
                            switch_spec.ch_names,
                            times=switch_spec.times,
                            freqs=switch_spec.freqs,
                            channels_per_page=60,
                            grid_shape=(6, 10),
                            cmap='gray',
                            title_prefix=f"{sub} ",
                            log_freq=True,
                            show=False)

        # Save each page as a separate figure file:
        for i, fig in enumerate(congruency_mask_pages):
            if rescaled:
                fig_name = f"{sub}_inc-con_sig_wavelet_clusters_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_no_freq_clusters_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_inc-con_sig_wavelet_clusters_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_no_freq_clusters_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        for i, fig in enumerate(switch_type_mask_pages):
            if rescaled:
                fig_name = f"{sub}_switch-repeat_sig_wavelet_no_across_freq_clusters_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_no_freq_clusters_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_switch-repeat_sig_wavelet_no_across_freq_clusters_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_no_freq_clusters_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        # get the mean differences themselves and plot them
        mean_diff_inc_vs_con = mean_diff(incongruent_spec._data, congruent_spec._data, axis=0)
        mean_diff_switch_vs_repeat = mean_diff(switch_spec._data, repeat_spec._data, axis=0)

        # Now, plot the mean differences directly:
        congruency_mean_diff_pages = plot_mask_pages(
            mean_diff_inc_vs_con,
            incongruent_spec.ch_names,
            times=incongruent_spec.times,
            freqs=incongruent_spec.freqs,
            grid_shape=(6, 10),
            cmap=parula_map,  # play with color maps
            title_prefix=f"{sub} Mean Inc-Con Diff: ",
            log_freq=True,
            show=False
        )

        switch_type_mean_diff_pages = plot_mask_pages(
            mean_diff_switch_vs_repeat,
            switch_spec.ch_names,
            times=switch_spec.times,
            freqs=switch_spec.freqs,
            grid_shape=(6, 10),
            cmap=parula_map,  # play with color maps
            title_prefix=f"{sub} Mean switch-repeat Diff: ",
            log_freq=True,
            show=False
        )

        # Save each page as a separate figure file:
        for i, fig in enumerate(congruency_mean_diff_pages):
            if rescaled:
                fig_name = f"{sub}_inc-con_mean_diff_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_inc-con_mean_diff_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        # Save each page as a separate figure file:
        for i, fig in enumerate(switch_type_mean_diff_pages):
            if rescaled:
                fig_name = f"{sub}_switch-repeat_mean_diff_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_switch-repeat_mean_diff_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)


Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


...

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_s25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  11 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  13 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  14 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  15 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Do

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_i25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...


[Parallel(n_jobs=1)]: Done 8049 tasks      | elapsed:  3.1min
[Parallel(n_jobs=1)]: Done 8050 tasks      | elapsed:  3.1min
[Parallel(n_jobs=1)]: Done 8050 tasks      | elapsed:  3.1min


Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_s25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Saved figure: D0057_inc-con_sig_wavelet_clusters_Stimulus_i25and75_fixationCrossBase_0.5sec-Stimulus_c25and75_fixationCrossBase_0.5sec_no_freq_clusters_uncorrected_page_1.jpg
Saved figure: D0057_inc-con_sig_wavelet_clusters_Stimulus_i25and75_fixationCrossBase_0.5sec-Stimulus_c25and75_fixationCrossBase_0.5sec_no_freq_clusters_uncorrected_page_2.jpg
Saved figure: D0057_inc-con_sig_wavelet_clusters_Stimulus_i25and75_fixationCrossBase_0.5sec-Stimulus_c25and75_fixationCrossBase_0.5sec_no_freq_clusters_uncorrected_page_3.jp

do the ratio too (while ignoring the across freq clusters)

In [5]:
times = [-0.5, 1.5]

# For loading precomputed wavelets, use output names from the configuration.
# Here, 'rescaled' toggles whether to load baseline-corrected (rescaled) TFRs.
rescaled = False  # Set to True if you wish to use baseline-corrected (dB scaled) wavelets

sig_wavelet_differences_per_subject = {}

for sub in subjects:

    sig_wavelet_differences_per_subject[sub] = {}
    good = get_good_data(sub, layout)
    RTs, skipped = calculate_RTs(good)
    avg_RT = np.median(RTs)
    print(f"Subject {sub} average RT: {avg_RT}")

    # Use the ratio function to load TFRs, compute their ratio in dB, and test significance
    congruency_ratio_mask, congruency_ratio_pvals, congruency_ratio_spec = load_and_get_sig_wavelet_ratio_differences(
        sub, layout,
        conditions_and_output_names_and_events['incongruent']['output_name'],
        conditions_and_output_names_and_events['congruent']['output_name'],
        rescaled,
        stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=(1,2), n_perm=100, n_jobs=1
    )

    switch_type_ratio_mask, switch_type_ratio_pvals, switch_type_ratio_spec = load_and_get_sig_wavelet_ratio_differences(
        sub, layout,
        conditions_and_output_names_and_events['switch']['output_name'],
        conditions_and_output_names_and_events['repeat']['output_name'],
        rescaled,
        stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=(1,2), n_perm=100, n_jobs=1
    )


    sig_wavelet_differences_per_subject[sub]['congruency'] = (congruency_ratio_mask, congruency_ratio_pvals)
    sig_wavelet_differences_per_subject[sub]['switch_type'] = (switch_type_ratio_mask, switch_type_ratio_pvals)

    # Plot the significant cluster mask pages (for the ratio test)
    congruency_ratio_mask_pages = plot_mask_pages(
        congruency_ratio_mask,
        congruency_ratio_spec.ch_names,
        times=congruency_ratio_spec.times,
        freqs=congruency_ratio_spec.freqs,
        channels_per_page=60,
        grid_shape=(6, 10),
        cmap=parula_map,
        title_prefix=f"{sub} ",
        log_freq=True,
        show=False
    )

    switch_type_ratio_mask_pages = plot_mask_pages(
        switch_type_ratio_mask,
        switch_type_ratio_spec.ch_names,
        times=switch_type_ratio_spec.times,
        freqs=switch_type_ratio_spec.freqs,
        channels_per_page=60,
        grid_shape=(6, 10),
        cmap=parula_map,
        title_prefix=f"{sub} ",
        log_freq=True,
        show=False
    )

    for i, fig in enumerate(congruency_ratio_mask_pages):
        if rescaled:
            fig_name = f"{sub}_inc_vs_con_sig_wavelet_no_across_freq_clusters_ratio_{conditions_and_output_names_and_events['incongruent']['output_name']}_vs_{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
        else:
            fig_name = f"{sub}_inc_vs_con_sig_wavelet_no_across_freq_clusters_ratio_{conditions_and_output_names_and_events['incongruent']['output_name']}_vs_{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
        fig_pathname = os.path.join(save_dir, fig_name)
        fig.savefig(fig_pathname, bbox_inches='tight')
        print("Saved figure:", fig_name)

    for i, fig in enumerate(switch_type_ratio_mask_pages):
        if rescaled:
            fig_name = f"{sub}_switch_vs_repeat_sig_wavelet_no_across_freq_clusters_ratio_{conditions_and_output_names_and_events['switch']['output_name']}_vs_{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_page_{i+1}.jpg"
        else:
            fig_name = f"{sub}_switch_vs_repeat_sig_wavelet_no_across_freq_clusters_ratio_{conditions_and_output_names_and_events['switch']['output_name']}_vs_{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_page_{i+1}.jpg"
        fig_pathname = os.path.join(save_dir, fig_name)
        fig.savefig(fig_pathname, bbox_inches='tight')
        print("Saved figure:", fig_name)

    # also plot the congruency and switch type ratio spectrograms themselves
    avg_congruency_ratio_spec = congruency_ratio_spec.average(lambda x: np.nanmean(x, axis=0), copy=True)
    figs = chan_grid(avg_congruency_ratio_spec, size=(20, 10), vmin=-2, vmax=2, cmap=parula_map, show=False)
    for i, fig in enumerate(figs):
        if rescaled:
            fig_name = f"{sub}_inc_vs_con_wavelet_ratio_spec_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
        else:
            fig_name = f"{sub}_inc_vs_con_wavelet_ratio_spec_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

    avg_switch_type_ratio_spec = switch_type_ratio_spec.average(lambda x: np.nanmean(x, axis=0), copy=True)
    figs = chan_grid(avg_switch_type_ratio_spec, size=(20, 10), vmin=-2, vmax=2, cmap=parula_map, show=False)
    for i, fig in enumerate(figs):
        if rescaled:
            fig_name = f"{sub}_switch_vs_repeat_wavelet_ratio_spec_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_page_{i+1}.jpg"
        else:
            fig_name = f"{sub}_switch_vs_repeat_wavelet_ratio_spec_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)


Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS

...

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_s25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected-tfr.h5 ...


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  11 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  13 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  14 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  15 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Do

Saved figure: D0057_inc_vs_con_sig_wavelet_no_across_freq_clusters_ratio_Stimulus_i25and75_fixationCrossBase_0.5sec_vs_Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_page_1.jpg
Saved figure: D0057_inc_vs_con_sig_wavelet_no_across_freq_clusters_ratio_Stimulus_i25and75_fixationCrossBase_0.5sec_vs_Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_page_2.jpg
Saved figure: D0057_inc_vs_con_sig_wavelet_no_across_freq_clusters_ratio_Stimulus_i25and75_fixationCrossBase_0.5sec_vs_Stimulus_c25and75_fixationCrossBase_0.5sec_uncorrected_page_3.jpg
Saved figure: D0057_switch_vs_repeat_sig_wavelet_no_across_freq_clusters_ratio_Stimulus_s25and75_fixationCrossBase_0.5sec_vs_Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected_page_1.jpg
Saved figure: D0057_switch_vs_repeat_sig_wavelet_no_across_freq_clusters_ratio_Stimulus_s25and75_fixationCrossBase_0.5sec_vs_Stimulus_r25and75_fixationCrossBase_0.5sec_uncorrected_page_2.jpg
Saved figure: D0057_switch_vs_repeat_sig_wavelet_no_across_freq

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\ensemble.py:127: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, frameon=False,


No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No basel

In [7]:
plt.close('all')

multitaper

In [ ]:
times = [-0.5, 1.5]
freqs = np.arange(2, 200., 2.)
n_cycles = freqs / 2
return_itc = False
time_bandwidth = 10 
make_multitaper = True

save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'multitaper', 'figs')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

if make_multitaper:
    # For making wavelets, extract events directly from the configuration dictionary.
    incongruent_events = conditions_and_output_names_and_events['incongruent']['events']
    congruent_events   = conditions_and_output_names_and_events['congruent']['events']
    switch_events      = conditions_and_output_names_and_events['switch']['events']
    repeat_events      = conditions_and_output_names_and_events['repeat']['events']

    sig_multitaper_differences_per_subject = {}

    for sub in subjects:

        sig_multitaper_differences_per_subject[sub] = {}
        # Preprocess and compute average reaction time (if desired)
        good = get_good_data(sub, layout)
        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)
        print(f"Subject {sub} average RT: {avg_RT}")

        # do inc-con, and also switch-repeat
        congruency_mask, congruency_pvals = make_and_get_sig_multitaper_differences(
            sub, layout, incongruent_events, congruent_events, times,
            stat_func=mean_diff, freqs=freqs, n_cycles=n_cycles, time_bandwidth=time_bandwidth, return_itc=return_itc, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)
            
        switch_type_mask, switch_type_pvals = make_and_get_sig_multitaper_differences(
            sub, layout, switch_events, repeat_events, times,
            stat_func=mean_diff, freqs=freqs, n_cycles=n_cycles, time_bandwidth=time_bandwidth, return_itc=return_itc, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)

        sig_multitaper_differences_per_subject[sub]['congruency'] = (congruency_mask, congruency_pvals)
        sig_multitaper_differences_per_subject[sub]['switch_type'] = (switch_type_mask, switch_type_pvals)

else:
    # For loading precomputed wavelets, use output names from the configuration.
    # Here, 'rescaled' toggles whether to load baseline-corrected (rescaled) TFRs.
    rescaled = False  # Set to True if you wish to use baseline-corrected (dB scaled) wavelets

    sig_multitaper_differences_per_subject = {}

    for sub in subjects:

        sig_multitaper_differences_per_subject[sub] = {}
        good = get_good_data(sub, layout)
        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)
        print(f"Subject {sub} average RT: {avg_RT}")

        # do inc-con, and also switch-repeat
        congruency_mask, congruency_pvals = load_and_get_sig_multitaper_differences(
            sub, layout,
            conditions_and_output_names_and_events['incongruent']['output_name'],
            conditions_and_output_names_and_events['congruent']['output_name'],
            rescaled,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)
        switch_type_mask, switch_type_pvals = load_and_get_sig_multitaper_differences(
            sub, layout,
            conditions_and_output_names_and_events['switch']['output_name'],
            conditions_and_output_names_and_events['repeat']['output_name'],
            rescaled,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)

        sig_multitaper_differences_per_subject[sub]['congruency'] = (congruency_mask, congruency_pvals)
        sig_multitaper_differences_per_subject[sub]['switch_type'] = (switch_type_mask, switch_type_pvals)

        # plot the sig cluster masks
        # Assume mask has shape (n_channels, ...) and you have channel names:
        # For example, you can get channel names from one of your TFR objects.
        # (Here we load one TFR to extract channel names.)
        incongruent_spec = load_multitaper(sub, layout, conditions_and_output_names_and_events['incongruent']['output_name'], rescaled)
        congruent_spec = load_multitaper(sub, layout, conditions_and_output_names_and_events['congruent']['output_name'], rescaled)
        switch_spec = load_multitaper(sub, layout, conditions_and_output_names_and_events['switch']['output_name'], rescaled)
        repeat_spec = load_multitaper(sub, layout, conditions_and_output_names_and_events['repeat']['output_name'], rescaled)

        ch_names = incongruent_spec.ch_names  # list of channel names

        # Now plot the mask pages:
        congruency_mask_pages = plot_mask_pages(congruency_mask,
                            incongruent_spec.ch_names,
                            times=incongruent_spec.times,
                            freqs=incongruent_spec.freqs,
                            channels_per_page=60,
                            grid_shape=(6, 10),
                            cmap=parula_map,
                            title_prefix=f"{sub} ",
                            log_freq=True,
                            show=False)
        
        switch_type_mask_pages = plot_mask_pages(switch_type_mask,
                            switch_spec.ch_names,
                            times=switch_spec.times,
                            freqs=switch_spec.freqs,
                            channels_per_page=60,
                            grid_shape=(6, 10),
                            cmap=parula_map,
                            title_prefix=f"{sub} ",
                            log_freq=True,
                            show=False)

        # Save each page as a separate figure file:
        for i, fig in enumerate(congruency_mask_pages):
            if rescaled:
                fig_name = f"{sub}_inc-con_sig_multitaper_clusters_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_inc-con_sig_multitaper_clusters_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        for i, fig in enumerate(switch_type_mask_pages):
            if rescaled:
                fig_name = f"{sub}_switch-repeat_sig_multitaper_clusters_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_switch-repeat_sig_multitaper_clusters_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        # get the mean differences themselves and plot them
        mean_diff_inc_vs_con = mean_diff(incongruent_spec._data, congruent_spec._data, axis=0)
        mean_diff_switch_vs_repeat = mean_diff(switch_spec._data, repeat_spec._data, axis=0)

        # Now, plot the mean differences directly:
        congruency_mean_diff_pages = plot_mask_pages(
            mean_diff_inc_vs_con,
            incongruent_spec.ch_names,
            times=incongruent_spec.times,
            freqs=incongruent_spec.freqs,
            grid_shape=(6, 10),
            cmap=parula_map,  # play with color maps
            title_prefix=f"{sub} Mean Inc-Con Diff: ",
            log_freq=True,
            show=False
        )

        switch_type_mean_diff_pages = plot_mask_pages(
            mean_diff_switch_vs_repeat,
            switch_spec.ch_names,
            times=switch_spec.times,
            freqs=switch_spec.freqs,
            grid_shape=(6, 10),
            cmap=parula_map,  # play with color maps
            title_prefix=f"{sub} Mean switch-repeat Diff: ",
            log_freq=True,
            show=False
        )

        # Save each page as a separate figure file:
        for i, fig in enumerate(congruency_mean_diff_pages):
            if rescaled:
                fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        # Save each page as a separate figure file:
        for i, fig in enumerate(switch_type_mean_diff_pages):
            if rescaled:
                fig_name = f"{sub}_switch-repeat_mean_diff_multitaper_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_rescaled_page_{i+1}.jpg"
            else:
                fig_name = f"{sub}_switch-repeat_mean_diff_multitaper_{conditions_and_output_names_and_events['switch']['output_name']}-{conditions_and_output_names_and_events['repeat']['output_name']}_uncorrected_page_{i+1}.jpg"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)
